In [ ]:
from aim2dat.strct import Structure

strct_molecule = Structure(
    label="molecule",
    elements=["N", "H", 1, 1],
    positions=[
        [0.000000, 0.000000, 0.000000],
        [0.000000, 0.000000, 1.008000],
        [0.950353, 0.000000, -0.336000],
        [-0.475176, -0.823029, -0.336000],
    ],
    pbc=False,
    site_attributes = {"oxidation_state": [-3, 1, 1, 1]}
)

strct_crystal = Structure(
    elements=["Ga", "As"],
    positions=[
        [0.0, 0.0, 0.0],
        [1.75, 1.75, 1.75],
    ],
    cell=[
        [0.0, 4.066, 4.0660001],
        [4.066, 0.0, 4.066],
        [4.066, 4.066, 0.0],
    ],
    is_cartesian=False,
    wrap=True,
    pbc=[True, True, True],
)

In [ ]:
print(strct_crystal)

In [ ]:
Structure.import_methods

In [ ]:
from ase.spacegroup import crystal

a = 4.066 * 2.0
GaAs_conv = crystal(
    ("Ga", "As"),
    basis=((0.0, 0.0, 0.0), (0.75, 0.75, 0.75)),
    spacegroup=216,
    cellpar=[a, a, a, 90, 90, 90],
    primitive_cell=False,
)

strct_crystal_conv = Structure.from_ase_atoms(GaAs_conv, label="GaAs_crystal")
print(strct_crystal_conv)

In [ ]:
Structure.export_methods

In [ ]:
strct_crystal_conv.to_file("GaAs_crystal.xsf")

In [ ]:
strct_molecule.analysis_methods

By calling the function the analysis is performed and the results returned. As an example we determine the space group of the crystal using the <a href="https://spglib.readthedocs.io/en/latest/" target="_blank">spglib</a> package as backend:

In [ ]:
strct_crystal.determine_space_group()

In [ ]:
print("Attribute:", strct_crystal.attributes["space_group"])
print("Extra:", strct_crystal.extras["space_group"])

In [ ]:
strct_molecule.manipulation_methods

In [ ]:
strct_crystal_AsAl = strct_crystal_conv.substitute_elements(elements=["Ga", "Al"])
print(strct_crystal_AsAl)

If the element is not part of the structure, no substitution can be conducted and the original object is returned

In [ ]:
strct_crystal_GaAs = strct_crystal_conv.substitute_elements(elements=["C", "Al"])
print(
    "Both structure objects have the same id and remain unchanged:",
    id(strct_crystal_GaAs) == id(strct_crystal_conv),
)
print(strct_crystal_GaAs)

In [ ]:
from aim2dat.strct.ext_analysis import calculate_prdf

prdf = calculate_prdf(strct_crystal_GaAs, r_max=7.5)

In [ ]:
strct_crystal_GaAs.extras.keys()

In [ ]:
from aim2dat.strct.ext_analysis.decorator import external_analysis_method
from aim2dat.utils.element_properties import get_element_symbol


@external_analysis_method
def determine_n_element(structure, element="H"):
    element = get_element_symbol(element)
    if element in structure.chem_formula:
        return (None, structure.chem_formula[element])
    else:
        return (None, 0)


determine_n_element(strct_crystal_GaAs, "Ga")
strct_crystal_GaAs.extras["n_element"]

In [ ]:
from aim2dat.strct.ext_manipulation.decorator import external_manipulation_method
from aim2dat.utils.element_properties import get_element_symbol

@external_manipulation_method
def add_atom(structure, element, position, change_label=False):
    element = get_element_symbol(element)
    if element in structure.elements:
        new_structure = structure.to_dict()
        new_structure["elements"] = list(new_structure["elements"]) + [element]
        new_structure["positions"] = list(new_structure["positions"]) + [position]
        if new_structure["kinds"] is not None:
            new_structure["kinds"] = list(new_structure["kinds"]) + [None]
        for site_attr, val in new_structure["site_attributes"].items():
            new_structure["site_attributes"][site_attr] = list(val) + [None]
        return new_structure, "_add"

new_structure = add_atom(strct_molecule, "O", [3.0, 3.0, 3.0], change_label=True)
print(new_structure)
new_structure = add_atom(strct_molecule, "H", [3.0, 3.0, 3.0], change_label=True)
print(new_structure)